In [1]:
from typing import Optional, Union
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier
from lightgbm import LGBMClassifier, LGBMRegressor

#from lightgbm import LGBMClassifier, LGBMRegressor


class HurdleRegression(BaseEstimator):
    """ Regression model which handles excessive zeros by fitting a two-part model and combining predictions:
            1) binary classifier
            2) continuous regression
    Implementeted as a valid sklearn estimator, so it can be used in pipelines and GridSearch objects.
    Args:
        clf_name: currently supports either 'logistic' or 'LGBMClassifier'
        reg_name: currently supports either 'linear' or 'LGBMRegressor'
        clf_params: dict of parameters to pass to classifier sub-model when initialized
        reg_params: dict of parameters to pass to regression sub-model when initialized
    """

    def __init__(self,
                 clf_name: str = 'logistic',
                 reg_name: str = 'linear',
                 clf_params: Optional[dict] = None,
                 reg_params: Optional[dict] = None):

        self.clf_name = clf_name
        self.reg_name = reg_name
        self.clf_params = clf_params
        self.reg_params = reg_params
        self.clf_fi = []
        self.reg_fi = []

    @staticmethod
    def _resolve_estimator(func_name: str):
        """ Lookup table for supported estimators.
        This is necessary because sklearn estimator default arguments
        must pass equality test, and instantiated sub-estimators are not equal. """

        funcs = {'linear': LinearRegression(),
                 'logistic': LogisticRegression(solver='liblinear'),
                 'LGBMRegressor': LGBMRegressor(n_estimators=250),
                 'LGBMClassifier': LGBMClassifier(n_estimators=250),
                 'RFRegressor': XGBRFRegressor(n_estimators=250,n_jobs=-2),
                 'RFClassifier': XGBRFClassifier(n_estimators=250,n_jobs=-2),
                 'GBMRegressor': GradientBoostingRegressor(n_estimators=200),
                 'GBMClassifier': GradientBoostingClassifier(n_estimators=200),
                 'XGBRegressor': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=-2),
                 'XGBClassifier': XGBClassifier(n_estimators=100,learning_rate=0.05,n_jobs=-2),
                 'HGBRegressor': HistGradientBoostingRegressor(max_iter=200),
                 'HGBClassifier': HistGradientBoostingClassifier(max_iter=200),
                }

        return funcs[func_name]

    def fit(self,
            X: Union[np.ndarray, pd.DataFrame],
            y: Union[np.ndarray, pd.Series]):
        X, y = check_X_y(X, y, dtype=None,
                         accept_sparse=False,
                         accept_large_sparse=False,
                         force_all_finite='allow-nan')

        if X.shape[1] < 2:
            raise ValueError('Cannot fit model when n_features = 1')

        self.clf_ = self._resolve_estimator(self.clf_name)
        if self.clf_params:
            self.clf_.set_params(**self.clf_params)
        self.clf_.fit(X, y > 0)
        self.clf_fi = self.clf_.feature_importances_

        self.reg_ = self._resolve_estimator(self.reg_name)
        if self.reg_params:
            self.reg_.set_params(**self.reg_params)
        self.reg_.fit(X[y > 0], y[y > 0])
        self.reg_fi = self.reg_.feature_importances_

        self.is_fitted_ = True
        return self


    def predict_bck(self, X: Union[np.ndarray, pd.DataFrame]):
        """ Predict combined response using binary classification outcome """
        X = check_array(X, accept_sparse=False, accept_large_sparse=False)
        check_is_fitted(self, 'is_fitted_')
        return self.clf_.predict(X) * self.reg_.predict(X)

    def predict(self, X: Union[np.ndarray, pd.DataFrame]):
        """ Predict combined response using probabilistic classification outcome """
        X = check_array(X, accept_sparse=False, accept_large_sparse=False)
        check_is_fitted(self, 'is_fitted_')
        return self.clf_.predict_proba(X)[:, 1] * self.reg_.predict(X)

    



In [5]:
hp_config = {
    "clf":{
        "learning_rate": 0.05,
        "n_estimators": 100,
        "n_jobs": 12
    },
    "reg":{
        "learning_rate": 0.05,
        "n_estimators": 100,
        "n_jobs": 12
    }
}
common_config = {
        "name": "lavender_haze",
        "algorithm": "HurdleRegression",
        "clf_name":"LGBMClassifier",
        "reg_name":"LGBMRegressor",
        "depvar": "ged_sb_dep",
        "queryset": "fatalities003_pgm_broad",
        "data_train": "baseline",
        "level": "pgm",
        'steps': [*range(1, 36 + 1, 1)],
}

In [6]:
cls_model = HurdleRegression(clf_name=common_config['clf_name'], reg_name=common_config['reg_name'], clf_params=hp_config['clf'], reg_params=hp_config['reg'])

In [7]:
cls_model

HurdleRegression(clf_name='LGBMClassifier',
                 clf_params={'learning_rate': 0.05, 'n_estimators': 100,
                             'n_jobs': 12},
                 reg_name='LGBMRegressor',
                 reg_params={'learning_rate': 0.05, 'n_estimators': 100,
                             'n_jobs': 12})

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../data/raw/raw.parquet")
df

tlag1_dr_mod_gs  spei1_gs_prev10_anom  \
month_id priogrid_gid                                          
1        62356                     0.0                   0.0   
         79599                     0.0                   0.0   
         79600                     0.0                   0.0   
         79601                     0.0                   0.0   
         80317                     0.0                   0.0   
...                                ...                   ...   
852      190496                    0.0                   0.0   
         190507                    0.0                   0.0   
         190508                    0.0                   0.0   
         190510                    0.0                   0.0   
         190511                    0.0                   0.0   

                       tlag_12_crop_sum  spei1gsy_lowermedian_count  \
month_id priogrid_gid                                                 
1        62356                      0.0                         0.0   
         79599                      0.0                         0.0   
         79600                      0.0                         0.0   
         79601                      0.0                         0.0   
         80317                      0.0                         0.0   
...                                 ...                         ...   
852      190496                     0.0                         0.0   
         190507                     0.0                         0.0   
         190508                     0.0                         0.0   
         190510                     0.0                         0.0   
         190511                     0.0                         0.0   

                       ln_ged_sb_dep  ged_sb  ged_os  ged_ns  treelag_1_sb  \
month_id priogrid_gid                                                        
1        62356                   0.0     0.0     0.0     0.0           0.0   
         79599                   0.0     0.0     0.0     0.0           0.0   
         79600                   0.0     0.0     0.0     0.0           0.0   
         79601                   0.0     0.0     0.0     0.0           0.0   
         80317                   0.0     0.0     0.0     0.0           0.0   
...                              ...     ...     ...     ...           ...   
852      190496                  0.0     0.0     0.0     0.0           0.0   
         190507                  0.0     0.0     0.0     0.0           0.0   
         190508                  0.0     0.0     0.0     0.0           0.0   
         190510                  0.0     0.0     0.0     0.0           0.0   
         190511                  0.0     0.0     0.0     0.0           0.0   

                       treelag_2_sb  ...  dist_diamsec  imr_mean  \
month_id priogrid_gid                ...                           
1        62356                  0.0  ...     19.235384       0.0   
         79599                  0.0  ...      3.640055     100.0   
         79600                  0.0  ...      3.807887     100.0   
         79601                  0.0  ...      4.031129     100.0   
         80317                  0.0  ...      3.000000     100.0   
...                             ...  ...           ...       ...   
852      190496                 0.0  ...     10.295630     150.0   
         190507                 0.0  ...      6.103278     419.0   
         190508                 0.0  ...      5.830952     419.0   
         190510                 0.0  ...      5.385165     419.0   
         190511                 0.0  ...      5.220153     419.0   

                       ln_ttime_mean  ln_bdist3  ln_capdist  ln_pop_gpw_sum  \
month_id priogrid_gid                                                         
1        62356              7.989464   2.263900    7.817437        0.000000   
         79599              5.251089   2.961998    7.187934        8.266445   
         79600              5.656525   0.364952   